In [254]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function

from datetime import date
import datetime
#imports

# Ideas

Raise error when len does not match before and after merging datasets (means that a player wasn't found -- likely because he is a junior or has two last names, etc.)

# To Do

1. Address last name issue
2. Address issue of list not being equal to 30
    - Really need to add TBD or something for the pitchers who are not listed yet
3. Get pitcher xba onto
4. Deal with doubleheaders

In [255]:
batters_df = pd.read_csv('../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../data/pitchers_teams.csv')

In [256]:
# duplicate_name_check = pitchers_df[['first_name', 'last_name']]

In [257]:
batters_df[batters_df['last_name'].str.contains(" ")]

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 15
5,Cruz Jr.,Nelson,443558,2022,18.2,10.6,0.181,0.264,0.500,0.359,0.344,0.236,90.2,6.9,10.8,NaN
19,Brantley Jr.,Michael,488726,2022,12.3,9.8,0.275,0.318,0.539,0.390,0.385,0.222,89.5,11.5,8.4,NaN
65,Springer III,George,543807,2022,21.4,6.9,0.278,0.273,0.491,0.354,0.340,0.219,90.8,16.2,10.0,NaN
95,Voit III,Luke,572228,2022,30.9,17.6,0.204,0.192,0.370,0.321,0.344,0.179,88.3,20.5,11.8,NaN
140,Bradley Jr.,Jackie,598265,2022,24.8,8.6,0.189,0.224,0.351,0.281,0.291,0.128,90.2,10.0,2.9,NaN
236,Mancini III,Trey,641820,2022,20.5,6.3,0.287,0.309,0.523,0.375,0.361,0.215,90.1,10.8,8.8,NaN
273,De La Cruz,Bryan,650559,2022,23.7,15.8,0.290,0.247,0.377,0.326,0.360,0.130,86.1,10.3,4.3,NaN
282,Mullins II,Cedric,656775,2022,18.9,5.4,0.269,0.262,0.478,0.345,0.328,0.216,90.1,19.4,10.3,NaN
297,Acuna Jr.,Ronald,660670,2022,37.0,13.0,0.282,0.280,0.656,0.432,0.390,0.376,94.0,3.1,27.3,NaN
315,Rivas III,Alfonso,663845,2022,28.0,16.0,0.262,0.223,0.377,0.316,0.350,0.155,87.5,-2.4,7.1,NaN


In [258]:
len(batters_df)

399

In [259]:
# bat_xbadiff = batters_df[['last_name', 'first_name', 'xbadiff','batting_avg', 'xba']]
#Make a table with just the xBA difference

In [260]:
# pitch_xbadiff = pitchers_df[['last_name', 'first_name', 'xbadiff', 'batting_avg', 'xba']]
# pitch_xbadiff.sort_values('xba', ascending=False)

# Probable Pitchers

In [261]:
d = datetime.datetime.now()
today = d.strftime("%Y-%m-%d")
#Get today's date

In [262]:
url = f'https://www.mlb.com/probable-pitchers/{today}'
#URL of where I will get the pitchers' names

In [263]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [264]:
# pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})

In [265]:
pitchers = soup.find_all('a', {'class': 'probable-pitchers__pitcher-name-link'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

#Important to note that if a pitcher is TBD then he is skipped, so need to figure out how to account for that

In [266]:
probable_pitchers = [i.split('">')[1] for i in probable_pitchers] 
#Getting only the pitchers' names

In [267]:
probable_pitchers

['George Kirby</a>',
 'Chris Bassitt</a>',
 'Kyle Hendricks</a>',
 'Zac Gallen</a>',
 'Michael Lorenzen</a>',
 'Adam Oller</a>',
 'Ranger Suarez</a>',
 'Julio Urias</a>',
 'Jakob Junis</a>',
 'Dakota Hudson</a>',
 'Sean Manaea</a>',
 'Charlie Morton</a>',
 'Jhonathan Diaz</a>',
 'Paul Blackburn</a>',
 'Bruce Zimmermann</a>',
 'Michael Pineda</a>',
 'Hyun Jin Ryu</a>',
 'Ryan Yarbrough</a>',
 'Eric Lauer</a>',
 'Trevor Rogers</a>',
 'Luis Castillo</a>',
 'Zach Thompson</a>',
 'Cristian Javier</a>',
 'Erick Fedde</a>',
 'Rich Hill</a>',
 'Glenn Otto</a>',
 'Shane Bieber</a>',
 'Devin Smeltzer</a>',
 'Jordan Montgomery</a>',
 'Dallas Keuchel</a>',
 'Carlos Hernandez</a>',
 'German Marquez</a>']

In [268]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [269]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [270]:
# probable_pitchers.insert(15, ['Albert', 'Abreu'])
#Temporary

In [271]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF

In [272]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [273]:
new_teams = soup.find_all('span', {'class':['probable-pitchers__team-name probable-pitchers__team-name--away', 'probable-pitchers__team-name probable-pitchers__team-name--home']})
teams_of_probable_pitchers = []
for i in new_teams:
    teams_of_probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [274]:
teams_of_probable_pitchers = [x.strip(' ') for x in teams_of_probable_pitchers]
#Getting only the pitchers' names

In [275]:
teams_of_probable_pitchers = [i.split('>\n')[1] for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [276]:
teams_of_probable_pitchers = teams_of_probable_pitchers = [x.strip('                             ') for x in teams_of_probable_pitchers]
#Stripping white space

In [277]:
teams_of_probable_pitchers = [i.split('\n')[0] for i in teams_of_probable_pitchers] 

In [278]:
len(teams_of_probable_pitchers)

32

In [279]:
df_probable_pitchers

,first_name,last_name,2
0,George,Kirby,None
1,Chris,Bassitt,None
2,Kyle,Hendricks,None
3,Zac,Gallen,None
4,Michael,Lorenzen,None
5,Adam,Oller,None
6,Ranger,Suarez,None
7,Julio,Urias,None
8,Jakob,Junis,None
9,Dakota,Hudson,None


In [280]:
df_probable_pitchers['team'] = teams_of_probable_pitchers
#Putting team on this way, now we just need to change it to the abbreviation

In [281]:
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Astros', 'HOU', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Nationals', 'WAS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Giants', 'SF', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Blue Jays', 'TOR', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Angels', 'LAA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'White Sox', 'CWS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Guardians', 'CLE', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Athletics', 'OAK', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Twins', 'MIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rays', 'TB', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mariners', 'SEA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Marlins', 'MIA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Padres', 'SD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Pirates', 'PIT', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Red Sox', 'BOS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Orioles', 'BAL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Braves', 'ATL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rangers', 'TEX', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Phillies', 'PHI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mets', 'NYM', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Yankees', 'NYY', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Royals', 'KC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cubs', 'CHC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Brewers', 'MIL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Reds', 'CIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rockies', 'COL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Tigers', 'DET', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Dodgers', 'LAD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'D-backs', 'ARI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cardinals', 'STL', df_probable_pitchers['team'])
#Changing team nickname to abbreviation so it will match the other DF -- Will clean this up in the future

In [282]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [283]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [284]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [285]:
misc_pitchers = misc_pitchers.rename(columns={"Team": "team"})
#Make this lowercase so it matches team column in probable pitchers

In [286]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'team']]
#reordering columns

# Get Batters' Teams

In [287]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [288]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [289]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [290]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [291]:
len(df_probable_pitchers)

32

In [292]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [293]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name', 'team'], how='left')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner')
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')
#Adding team to all of the players
#Joining probables as a left join so it will include all of the probables without exception -- may make one of their stats
# unavailable if they don't have enough PAs 

In [294]:
# ^^^^ FOR PROBABLE PITCHERS MATCH IT ON TEAM AS WELL. THAT WILL AVOID THE LUIS GARCIA SITUATION

In [295]:
# pitchers_df.loc[pitchers_df['last_name'] == 'Garcia']
#So somewhere it has duplicated Garcia

In [296]:
hello = pd.merge(df_probable_pitchers, pitchers_df)
#test

In [297]:
df_probable_pitchers = pd.merge(df_probable_pitchers, pitchers_df, on=['team', 'last_name', 'first_name'], how='left')
#Adding stats to probable pitchers

#This is what is adding the other Luis Garcia

# Which player to pick for BTS

In [298]:
pp_xba = df_probable_pitchers[['last_name', 'first_name', 'team', 'batting_avg', 'xba', 'xslg', 'p_bb_percent']]
#How can I compare this to batters df??

In [299]:
pp_xba.rename(columns= {'xba': 'pitcher_xba'})
#Eventually might move this to another df or merge, unclear

,last_name,first_name,team,batting_avg,pitcher_xba,xslg,p_bb_percent
0,Kirby,George,SEA,NaN,NaN,NaN,NaN
1,Bassitt,Chris,NYM,0.197,0.216,0.403,6.2
2,Hendricks,Kyle,CHC,0.245,0.291,0.494,8.0
3,Gallen,Zac,ARI,0.160,0.178,0.278,3.7
4,Lorenzen,Michael,LAA,0.221,0.244,0.453,8.7
5,Oller,Adam,OAK,0.357,0.339,0.755,15.4
6,Suarez,Ranger,PHI,0.274,0.282,0.472,9.1
7,Urias,Julio,LAD,0.225,0.231,0.392,6.6
8,Junis,Jakob,SF,0.192,0.225,0.464,5.4
9,Hudson,Dakota,STL,0.231,0.280,0.515,11.8


In [300]:
#Temporary
# pp_xba = pp_xba.drop([0])
# pp_xba = pp_xba.reset_index(drop=True)

One thing I could do is... switch the one column with the next, and then merge it with the batter's df
So for example if Oakland is playing Cleveland, I can switch the pitchers with each other. So Oakland's pitcher will be on Cleveland, and then I will have the pitcher xba to match with the team. Then I can basically set that for every member of the team, and then add that with the hitter xba and see if it meets the condition

In [301]:
pp_xba.sort_values(by='xslg', ascending=False)

,last_name,first_name,team,batting_avg,xba,xslg,p_bb_percent
5,Oller,Adam,OAK,0.357,0.339,0.755,15.4
15,Pineda,Michael,DET,0.250,0.304,0.584,4.8
30,Hernandez,Carlos,KC,0.337,0.313,0.549,11.2
31,Marquez,German,COL,0.328,0.308,0.524,6.8
21,Thompson,Zach,PIT,0.292,0.299,0.520,9.9
9,Hudson,Dakota,STL,0.231,0.280,0.515,11.8
26,Bieber,Shane,CLE,0.242,0.277,0.504,6.7
11,Morton,Charlie,ATL,0.273,0.290,0.495,12.7
2,Hendricks,Kyle,CHC,0.245,0.291,0.494,8.0
14,Zimmermann,Bruce,BAL,0.250,0.285,0.476,6.4


In [302]:
batters_df.loc[batters_df['Team'] == 'NYY']

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 15,Team
25,Gonzalez,Marwin,503556,2022,37.0,7.4,0.208,0.205,0.274,0.251,0.269,0.069,93.9,14.7,0.0,NaN,NYY
32,Donaldson,Josh,518626,2022,25.6,13.2,0.233,0.260,0.493,0.371,0.370,0.233,92.4,12.4,13.9,NaN,NYY
36,LeMahieu,DJ,518934,2022,13.0,8.9,0.282,0.307,0.489,0.375,0.381,0.182,89.6,6.0,6.4,NaN,NYY
38,Rizzo,Anthony,519203,2022,16.5,11.3,0.232,0.275,0.572,0.402,0.389,0.297,91.7,15.7,12.2,NaN,NYY
39,Stanton,Giancarlo,519317,2022,29.1,6.3,0.284,0.321,0.661,0.443,0.376,0.340,98.4,11.3,26.8,NaN,NYY
55,Hicks,Aaron,543305,2022,21.2,17.3,0.217,0.238,0.338,0.323,0.377,0.100,88.5,13.1,1.6,NaN,NYY
56,Higashioka,Kyle,543309,2022,23.3,5.0,0.170,0.254,0.417,0.307,0.301,0.163,94.3,16.7,9.5,NaN,NYY
105,Judge,Aaron,592450,2022,27.6,10.4,0.305,0.324,0.778,0.478,0.397,0.454,96.7,13.0,29.3,NaN,NYY
161,Gallo,Joey,608336,2022,37.5,12.5,0.202,0.226,0.554,0.366,0.323,0.328,91.0,17.4,20.8,NaN,NYY
201,Trevino,Jose,624431,2022,12.5,6.3,0.178,0.278,0.398,0.313,0.323,0.121,86.1,12.8,2.6,NaN,NYY


In [303]:
pp_xba['Team'] = " "
#New column to house when I switch the teams with each other
#I'm doing this so I can eventually merge it onto the batters' df with the hitters' team

C:\Users\User\AppData\Local\Temp/ipykernel_5048/1964202557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'] = " "


# Getting Pitcher Stats onto Batter DF

FOR NOW, UNTIL I CLEAN THE BELOW UP A LITTLE MORE AND HAVE IT ITERATE THROUGH AND ADJUST TO THE NUMBER OF ROWS PER DAY, JUST USE THE AMOUNT OF ROWS THAT THE LEN() SHOWS AND COMMENT OUT THE REST

In [304]:
pp_xba['Team'][0] = pp_xba['team'][1]
pp_xba['Team'][1] = pp_xba['team'][0]
pp_xba['Team'][2] = pp_xba['team'][3]
pp_xba['Team'][3] = pp_xba['team'][2]
pp_xba['Team'][4] = pp_xba['team'][5]
pp_xba['Team'][5] = pp_xba['team'][4]
pp_xba['Team'][6] = pp_xba['team'][7]
pp_xba['Team'][7] = pp_xba['team'][6]
pp_xba['Team'][8] = pp_xba['team'][9]
pp_xba['Team'][9] = pp_xba['team'][8]
pp_xba['Team'][10] = pp_xba['team'][11]
pp_xba['Team'][11] = pp_xba['team'][10]
pp_xba['Team'][12] = pp_xba['team'][13]
pp_xba['Team'][13] = pp_xba['team'][12]
pp_xba['Team'][14] = pp_xba['team'][15]
pp_xba['Team'][15] = pp_xba['team'][14]
pp_xba['Team'][16] = pp_xba['team'][17]
pp_xba['Team'][17] = pp_xba['team'][16]
pp_xba['Team'][18] = pp_xba['team'][19]
pp_xba['Team'][19] = pp_xba['team'][18]
pp_xba['Team'][20] = pp_xba['team'][21]
pp_xba['Team'][21] = pp_xba['team'][20]
pp_xba['Team'][22] = pp_xba['team'][23]
pp_xba['Team'][23] = pp_xba['team'][22]
pp_xba['Team'][24] = pp_xba['team'][25]
pp_xba['Team'][25] = pp_xba['team'][24]
pp_xba['Team'][26] = pp_xba['team'][27]
pp_xba['Team'][27] = pp_xba['team'][26]
pp_xba['Team'][28] = pp_xba['team'][29]
pp_xba['Team'][29] = pp_xba['team'][28]
pp_xba['Team'][30] = pp_xba['team'][31]
pp_xba['Team'][31] = pp_xba['team'][30]

C:\Users\User\AppData\Local\Temp/ipykernel_5048/279755331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][0] = pp_xba['team'][1]
C:\Users\User\AppData\Local\Temp/ipykernel_5048/279755331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][1] = pp_xba['team'][0]
C:\Users\User\AppData\Local\Temp/ipykernel_5048/279755331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][2] = pp_xba['team'

In [305]:
pp_xba

,last_name,first_name,team,batting_avg,xba,xslg,p_bb_percent,Team
0,Kirby,George,SEA,NaN,NaN,NaN,NaN,NYM
1,Bassitt,Chris,NYM,0.197,0.216,0.403,6.2,SEA
2,Hendricks,Kyle,CHC,0.245,0.291,0.494,8.0,ARI
3,Gallen,Zac,ARI,0.160,0.178,0.278,3.7,CHC
4,Lorenzen,Michael,LAA,0.221,0.244,0.453,8.7,OAK
5,Oller,Adam,OAK,0.357,0.339,0.755,15.4,LAA
6,Suarez,Ranger,PHI,0.274,0.282,0.472,9.1,LAD
7,Urias,Julio,LAD,0.225,0.231,0.392,6.6,PHI
8,Junis,Jakob,SF,0.192,0.225,0.464,5.4,STL
9,Hudson,Dakota,STL,0.231,0.280,0.515,11.8,SF


In [306]:
pp_xba.loc[0,'Team']

'NYM'

In [139]:
pp_xba = pp_xba.rename(columns={"batting_avg": "pitcher_batting_avg", "xba": "pitcher_xba"})
#Renaming columns for clarity

In [140]:
pp_xba

,last_name,first_name,team,pitcher_batting_avg,pitcher_xba,xslg,p_bb_percent,Team
0,Javier,Cristian,HOU,0.153,0.196,0.388,7.4,WAS
1,Fedde,Erick,WAS,0.246,0.278,0.419,11.5,HOU
2,Hill,Rich,BOS,0.213,0.236,0.472,8.0,TEX
3,Otto,Glenn,TEX,NaN,NaN,NaN,NaN,BOS
4,Kirby,George,SEA,NaN,NaN,NaN,NaN,NYM
5,Bassitt,Chris,NYM,0.197,0.216,0.403,6.2,SEA
6,Bieber,Shane,CLE,0.242,0.277,0.504,6.7,MIN
7,Smeltzer,Devin,MIN,NaN,NaN,NaN,NaN,CLE
8,Montgomery,Jordan,NYY,0.212,0.249,0.371,4.0,CWS
9,Keuchel,Dallas,CWS,0.340,0.291,0.449,11.2,NYY


In [67]:
# pp_xba = pp_xba.drop([8])
#temporary

In [68]:
batters_df = pd.merge(batters_df, pp_xba[['Team','pitcher_xba']],on='Team', how='left')
#Getting pitcher data onto batters df

In [69]:
xba_sum = batters_df['xba'] + batters_df['pitcher_xba']
#Adding sum 

In [70]:
batters_df['xba_total'] = xba_sum
#Adding xba sum into the batters dataframe

# Which Hitters to pick for BTS

# Best Picks for Today

In [71]:
picky = batters_df[(batters_df['xba'] > .300) & (batters_df['pitcher_xba'] > .300) & (batters_df['xba_total'] > .650)]
picky

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 15,Team,pitcher_xba,xba_total
77,Trout,Mike,545361,2022,24.0,16.5,0.323,0.313,0.779,0.491,0.438,0.465,92.9,22.5,27.1,NaN,LAA,0.339,0.652
144,Daza,Yonathan,602074,2022,7.0,3.5,0.365,0.346,0.506,0.400,0.396,0.160,85.1,5.4,6.1,NaN,COL,0.313,0.659
298,Olivares,Edward,658668,2022,13.2,5.3,0.371,0.355,0.568,0.414,0.406,0.212,92.7,0.6,6.7,NaN,KC,0.308,0.663


In [72]:
gods = batters_df[batters_df['xba_total'] > .625]
gods

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,xwoba,xobp,xiso,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 15,Team,pitcher_xba,xba_total
77,Trout,Mike,545361,2022,24.0,16.5,0.323,0.313,0.779,0.491,0.438,0.465,92.9,22.5,27.1,NaN,LAA,0.339,0.652
144,Daza,Yonathan,602074,2022,7.0,3.5,0.365,0.346,0.506,0.400,0.396,0.160,85.1,5.4,6.1,NaN,COL,0.313,0.659
193,Ward,Taylor,621493,2022,20.6,18.6,0.361,0.306,0.603,0.437,0.437,0.297,87.8,11.0,14.5,NaN,LAA,0.339,0.645
203,Reyes,Victor,622682,2022,16.0,8.0,0.261,0.394,0.568,0.432,0.442,0.174,91.1,3.6,5.3,NaN,DET,0.285,0.679
279,Castro,Willi,650489,2022,13.0,6.5,0.326,0.396,0.460,0.392,0.435,0.064,83.2,17.3,2.7,NaN,DET,0.285,0.681
298,Olivares,Edward,658668,2022,13.2,5.3,0.371,0.355,0.568,0.414,0.406,0.212,92.7,0.6,6.7,NaN,KC,0.308,0.663
310,Mountcastle,Ryan,663624,2022,24.8,3.4,0.268,0.333,0.556,0.384,0.362,0.224,91.5,14.9,10.8,NaN,BAL,0.304,0.637
376,Alvarez,Yordan,670541,2022,16.9,14.4,0.280,0.352,0.759,0.485,0.443,0.407,95.6,12.4,19.8,NaN,HOU,0.278,0.630
